#Request

In [28]:
import requests

csv_url = "https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv?departamento_nom=AMAZONAS&accessType=DOWNLOAD&api_foundry=true"
#csv_url = "./content/sample_data/sample_downloaded.csv"
root_path = "/content/drive/MyDrive/Salisbury/data/covid_downloaded.csv"
def downloadDataCsv(csv_url,root_path):
  req = requests.get(csv_url)
  url_content = req.content
  try:
      csv_file = open(root_path, 'wb')

      csv_file.write(url_content)

      csv_file.close()
      print("I finished")

  except:
      print("something was wrong")

#downloadDataCsv(csv_url,root_path)

#Model

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import math

import os
#!pip install sodapy -q
#!pip install socrata-py -q

#from sodapy import Socrata
#from socrata.authorization import Authorization


In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [27]:
def requestColombianData(limit = 1000):
  client = Socrata(
                  "www.datos.gov.co",
                  "idLwSviPi5dGMTd7GfIfW3CLU",
                  #username="sebastian_romerola@fet.edu.co",
                  #password="Sebastian#2018",
                  timeout=300
                  )
  results = client.get("gt2j-8ykr",
                      #  ciudad_municipio_nom="Neiva",
                      #  departamento_nom="HUILA",
                      limit=limit
                      )
  
  return pd.DataFrame.from_records(results)


def getCsvDataSet(path='/content/drive/MyDrive/Salisbury/data/covid_downloaded.csv'):
  return pd.read_csv(r''+path, low_memory=False)

In [4]:
#dataset from csv
df = getCsvDataSet()

#dataset from API
#df = None
#df = requestColombianData(460)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (14,17,19,20,22) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [5]:
df.head(20)

,fecha reporte web,ID de caso,Fecha de notificación,Código DIVIPOLA departamento,Nombre departamento,Código DIVIPOLA municipio,Nombre municipio,Edad,Unidad de medida de edad,Sexo,Tipo de contagio,Ubicación del caso,Estado,Código ISO del país,Nombre del país,Recuperado,Fecha de inicio de síntomas,Fecha de muerte,Fecha de diagnóstico,Fecha de recuperación,Tipo de recuperación,Pertenencia étnica,Nombre del grupo étnico
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380.0,ITALIA,Recuperado,27/2/2020 0:00:00,NaN,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6.0,NaN
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,4/3/2020 0:00:00,NaN,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5.0,NaN
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,29/2/2020 0:00:00,NaN,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6.0,NaN
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,8/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN
5,11/3/2020 0:00:00,6,10/3/2020 0:00:00,5,ANTIOQUIA,5360,ITAGUI,27,1,F,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
6,11/3/2020 0:00:00,7,8/3/2020 0:00:00,13001,CARTAGENA,13001,CARTAGENA,85,1,F,Importado,Casa,Leve,840.0,ESTADOS UNIDOS DE AMÉRICA,Recuperado,2/3/2020 0:00:00,NaN,11/3/2020 0:00:00,17/3/2020 0:00:00,PCR,6.0,NaN
7,11/3/2020 0:00:00,8,9/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,22,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6.0,NaN
8,11/3/2020 0:00:00,9,8/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,28,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,7/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN
9,12/3/2020 0:00:00,10,12/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,36,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,6/3/2020 0:00:00,NaN,12/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6.0,NaN


In [6]:
print(df.columns)
df.columns = [
  'web_date_report',
  'case_id',
  'notification_date',
  'davipola_departament_code',
  'department_name',
  'davipola_municipality_code',
  'municipality_name',
  'age',
  'age_measurement_unit',
  'sex',
  'contagion_type',
  'case_location',
  'state',
  'iso_country_code',
  'country_name',
  'recuperated',
  'beginning_symptoms_date',
  'date_of_death',
  'date_of_diagnosis',
  'date_of_recovery',
  'recovery_type',
  'ethnicity',
  'ethnic_group_name',
]


Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
       'Código DIVIPOLA departamento', 'Nombre departamento',
       'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad',
       'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
       'Ubicación del caso', 'Estado', 'Código ISO del país',
       'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas',
       'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación',
       'Tipo de recuperación', 'Pertenencia étnica',
       'Nombre del grupo étnico'],
      dtype='object')


In [7]:
df.astype = {  
  "web_date_report": np.str,
  "case_id": np.int, #ordinal
  "notification_date": np.str, 
  "davipola_departament_code": np.int, #categorical
  "department_name": np.str, #categorical
  "davipola_municipality_code": np.str, #categorical
  "municipality_name": np.str, #categorical
  "age": np.int, 
  "age_measurement_unit": np.int,
  "sex": np.str, #nominal
  "contagion_type": np.str, #categorical
  "case_location": np.str, #categorical
  "state": np.str, #ordinal
  "iso_country_code": np.int, #categorical
  "country_name": np.str, #categorical
  "recuperated": np.str, ##categorical
  "beginning_symptoms_date": np.str, #!/usr/bin/env python
  "date_of_diagnosis": np.str,
  "date_of_recovery": np.str,
  "recovery_type": np.str, #categorica
  "ethnicity": np.int, #nominal
  "ethnic_group_name": np.str, #nominal
  "date_of_death": np.str,
}

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4530610 entries, 0 to 4530609
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   web_date_report             object 
 1   case_id                     int64  
 2   notification_date           object 
 3   davipola_departament_code   int64  
 4   department_name             object 
 5   davipola_municipality_code  int64  
 6   municipality_name           object 
 7   age                         int64  
 8   age_measurement_unit        int64  
 9   sex                         object 
 10  contagion_type              object 
 11  case_location               object 
 12  state                       object 
 13  iso_country_code            float64
 14  country_name                object 
 15  recuperated                 object 
 16  beginning_symptoms_date     object 
 17  date_of_death               object 
 18  date_of_diagnosis           object 
 19  date_of_recovery     

In [9]:
df.describe()

,case_id,davipola_departament_code,davipola_municipality_code,age,age_measurement_unit,iso_country_code,ethnicity
count,4.530610e+06,4.530610e+06,4.530610e+06,4.530610e+06,4.530610e+06,3057.000000,4.471622e+06
mean,2.265345e+06,1.302291e+03,2.804815e+04,3.969038e+01,1.003440e+00,597.879293,5.907317e+00
std,1.307875e+06,5.853340e+03,2.598089e+04,1.788452e+01,6.372697e-02,285.800759,6.064583e-01
min,1.000000e+00,5.000000e+00,5.001000e+03,1.000000e+00,1.000000e+00,4.000000,1.000000e+00
25%,1.132693e+06,1.100000e+01,1.100100e+04,2.700000e+01,1.000000e+00,380.000000,6.000000e+00
50%,2.265346e+06,1.700000e+01,1.100100e+04,3.700000e+01,1.000000e+00,724.000000,6.000000e+00
75%,3.397998e+06,6.800000e+01,5.000100e+04,5.200000e+01,1.000000e+00,840.000000,6.000000e+00
max,4.530650e+06,4.700100e+04,9.977300e+04,1.140000e+02,3.000000e+00,1724.000000,6.000000e+00


In [10]:
np.shape(df)

(4530610, 23)

In [11]:
df.columns

Index(['web_date_report', 'case_id', 'notification_date',
       'davipola_departament_code', 'department_name',
       'davipola_municipality_code', 'municipality_name', 'age',
       'age_measurement_unit', 'sex', 'contagion_type', 'case_location',
       'state', 'iso_country_code', 'country_name', 'recuperated',
       'beginning_symptoms_date', 'date_of_death', 'date_of_diagnosis',
       'date_of_recovery', 'recovery_type', 'ethnicity', 'ethnic_group_name'],
      dtype='object')

uppercase to each str row

In [44]:
df.select_dtypes(include='object')

,web_date_report,notification_date,department_name,municipality_name,sex,contagion_type,case_location,state,country_name,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,date_of_recovery,recovery_type,ethnic_group_name
0,6/3/2020 0:00:00,2/3/2020 0:00:00,BOGOTA,BOGOTA,F,Importado,Casa,Leve,ITALIA,Recuperado,27/2/2020 0:00:00,0,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,0
1,9/3/2020 0:00:00,6/3/2020 0:00:00,VALLE,BUGA,M,Importado,Casa,Leve,ESPAÑA,Recuperado,4/3/2020 0:00:00,0,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,0
2,9/3/2020 0:00:00,7/3/2020 0:00:00,ANTIOQUIA,MEDELLIN,F,Importado,Casa,Leve,ESPAÑA,Recuperado,29/2/2020 0:00:00,0,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,0
3,11/3/2020 0:00:00,9/3/2020 0:00:00,ANTIOQUIA,MEDELLIN,M,Relacionado,Casa,Leve,0,Recuperado,6/3/2020 0:00:00,0,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,0
4,11/3/2020 0:00:00,9/3/2020 0:00:00,ANTIOQUIA,MEDELLIN,M,Relacionado,Casa,Leve,0,Recuperado,8/3/2020 0:00:00,0,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4530605,12/7/2021 0:00:00,29/6/2021 0:00:00,NORTE SANTANDER,CUCUTA,M,En estudio,Casa,Leve,0,Activo,25/6/2021 0:00:00,0,10/7/2021 0:00:00,0,0,0
4530606,12/7/2021 0:00:00,29/6/2021 0:00:00,BOLIVAR,SAN JUAN NEPOMUCENO,M,En estudio,Casa,Leve,0,Activo,25/6/2021 0:00:00,0,10/7/2021 0:00:00,0,0,0
4530607,12/7/2021 0:00:00,29/6/2021 0:00:00,NORTE SANTANDER,VILLA DEL ROSARIO,M,En estudio,Casa,Leve,0,Activo,25/6/2021 0:00:00,0,10/7/2021 0:00:00,0,0,0
4530608,12/7/2021 0:00:00,29/6/2021 0:00:00,NORTE SANTANDER,CUCUTA,M,En estudio,Casa,Leve,0,Activo,25/6/2021 0:00:00,0,10/7/2021 0:00:00,0,0,0


Se quitan los nulos y se reemplazan por 0

verifying nulls


```
print(df.isnull().values.any())
```
[source](https://towardsdatascience.com/a-complete-guide-to-principal-component-analysis-pca-in-machine-learning-664f34fc3e5a)

After use this function, the result of the code is false.


In [45]:
print(df.isnull().values.any())

False


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4530610 entries, 0 to 4530609
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   web_date_report             object 
 1   case_id                     int64  
 2   notification_date           object 
 3   davipola_departament_code   int64  
 4   department_name             object 
 5   davipola_municipality_code  int64  
 6   municipality_name           object 
 7   age                         int64  
 8   age_measurement_unit        int64  
 9   sex                         object 
 10  contagion_type              object 
 11  case_location               object 
 12  state                       object 
 13  iso_country_code            float64
 14  country_name                object 
 15  recuperated                 object 
 16  beginning_symptoms_date     object 
 17  date_of_death               object 
 18  date_of_diagnosis           object 
 19  date_of_recovery     

Similarity mesurement (radio basics funciotn)
1.  
*   ML / rbf -| knn graph / 
*   train to know about who would most highly survive
*   KNN graph (find the most near neibough
2.  
*   svm (support vector machine)





Adjency matrix. to build a graph after apply rbf and knn aproach?



In [26]:
df.head()

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name
0,2020-03-06,1,2020-03-02,11,BOGOTA,11001,BOGOTA,19,1,F,IMPORTADO,CASA,LEVE,380.0,ITALIA,RECUPERADO,2020-02-27,NaT,2020-03-06,2020-03-13,PCR,6.0,NaN
1,2020-03-09,2,2020-03-06,76,VALLE,76111,BUGA,34,1,M,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-03-04,NaT,2020-03-09,2020-03-19,PCR,5.0,NaN
2,2020-03-09,3,2020-03-07,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-02-29,NaT,2020-03-09,2020-03-15,PCR,6.0,NaN
3,2020-03-11,4,2020-03-09,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,2020-03-06,NaT,2020-03-11,2020-03-26,PCR,6.0,NaN
4,2020-03-11,5,2020-03-09,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,2020-03-08,NaT,2020-03-11,2020-03-23,PCR,6.0,NaN


In [12]:
df["beginning_symptoms_date"] = pd.to_datetime(df["beginning_symptoms_date"],format='%d/%m/%Y %H:%M:%S')
df["date_of_death"] = pd.to_datetime(df["date_of_death"],format='%d/%m/%Y %H:%M:%S')
df["web_date_report"] = pd.to_datetime(df["web_date_report"],format='%d/%m/%Y %H:%M:%S')
df["date_of_recovery"] = pd.to_datetime(df["date_of_recovery"],format='%d/%m/%Y %H:%M:%S')
df["date_of_diagnosis"] = pd.to_datetime(df["date_of_diagnosis"],format='%d/%m/%Y %H:%M:%S')
df["notification_date"] = pd.to_datetime(df["notification_date"],format='%d/%m/%Y %H:%M:%S')

###Does is first greather than second?

In [17]:
df["beginning_symptoms_date"][0] > df["beginning_symptoms_date"][1]

False

In [15]:
for i in df.select_dtypes(include='object'):
  df[i] = df[i].str.upper()

In [18]:
df.head()

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name
0,2020-03-06,1,2020-03-02,11,BOGOTA,11001,BOGOTA,19,1,F,IMPORTADO,CASA,LEVE,380.0,ITALIA,RECUPERADO,2020-02-27,NaT,2020-03-06,2020-03-13,PCR,6.0,NaN
1,2020-03-09,2,2020-03-06,76,VALLE,76111,BUGA,34,1,M,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-03-04,NaT,2020-03-09,2020-03-19,PCR,5.0,NaN
2,2020-03-09,3,2020-03-07,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-02-29,NaT,2020-03-09,2020-03-15,PCR,6.0,NaN
3,2020-03-11,4,2020-03-09,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,2020-03-06,NaT,2020-03-11,2020-03-26,PCR,6.0,NaN
4,2020-03-11,5,2020-03-09,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,2020-03-08,NaT,2020-03-11,2020-03-23,PCR,6.0,NaN


In [ ]:
df.describe()

Analisis de componentes principales (APC)

quitar lo que se puede.



In [41]:
#df = df.fillna(0)

desviación estandar de  para analizar causas de muerte

*italicized text*.fit

se realiza conversión a todos los que tengan

In [ ]:
#////////////////////////////////////////////////////////#
#                                                        /
#      Binarize Sex column values to avoid errors        /
#                                                        /
#////////////////////////////////////////////////////////#
from sklearn.preprocessing import Binarizer

#/-////////////////////////////////////////////////////////#
#                                                         /
#              average of recovery time per genre         /
#                                                         /
#/////////////////////////////////////////////////////////#

#////////////////////////////////////////////////////////#
#                                                        /
#     Capitalize Sex column values to avoid errors       /
#                                                        /
#////////////////////////////////////////////////////////#

#////////////////////////////////////////////////////////#
#                                                        /
#  substract betweeen two dates and store it the answer  /
#                                                        /
#////////////////////////////////////////////////////////#

def get_duration_of_illness(x:any):
  time = 0
  z = list()
  count = 0
  for e in x.index:
    count += 1
    if x.date_of_recovery[e] != 0:
      x.loc[e, "time_to_recover"] = abs(x.date_of_recovery[e] - x.beginning_symptoms_date[e])
    elif x.date_of_death[e] != 0:
      x.loc[e, "time_to_recover"] = abs(x.date_of_death[e] - x.beginning_symptoms_date[e])
    elif x.date_of_death[e] == 0 and x.date_of_recovery[e] == 0:
      print("NAN")
      x.loc[e, "time_to_recover"] = 0
  
  print("The total amount of data is: "+ str(count))

  return x

      # x.duration_of_illness.append()
      
      # abs((d2 - d1).days)

#////////////////////////////////////////////////////////#
#                                                        /
# Get important Columns to get the main of recovery time /
#                                                        /
#////////////////////////////////////////////////////////#

mean = 0

#it is not necessary use another array, a better approach may be use df directly
array_duration_of_illness = df.loc[
                                   :,
                                   [
                                    "sex",
                                    "beginning_symptoms_date",
                                    "date_of_recovery",
                                    "date_of_death",
                                    "recuperated"
                                    ]
                                   ].copy()

                                   #  sex M/ 15 avg to get recover from beginning_symptoms_date. 
                                   #  sex F/ 14 avg.

                                   # Male / Ibague / how long time do they neeed to get recover?
                                   #cound .. avg ... 

                                   ## share tje ML research results -| graph clustering. 
array_dates_women = df[df["sex"] == 'F']
get_duration_of_illness(array_dates_women)

array_dates_men = df[df["sex"] == "M"]
get_duration_of_illness(array_dates_men)

#////           For alive people
# df.beginning_symptoms_date - df.date_of_recovery 

#////           For death people
# df.beginning_symptoms_date - df.date_of_death 

In [ ]:
df.isnull().sum()

Replacing


In [ ]:
#df.boxplot('age','contagion_type',rot = 0,figsize=(5,6))

In [ ]:
df.dtypes.value_counts()

#Categorical Variables



Posible ways to replace
[Pandas Documentation 
](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html)
```
df.replace({'A': {0: 100, 4: 400}})
Result:
        A  B  C
0  100  5  a
1    1  6  b
2    2  7  c
3    3  8  d
4  400  9  e
```



# Data Transformation
Label or one hot enconding or replacing



1.   Delete departament name && municipality name 
2.   Binarize sex column
3.   Delete country name
4.   Normalize recovery type
5.   Write documentation


In [ ]:
#   df.replace([{'sex': {'F': 1, 'M': -1}]})
df.replace(['F',
            'M'],
          [ '1',
            '-1'])

In [ ]:

#   Female = 0; Male = 1

#   Female = 0; Male = 1
'''
#contagion type
df.replace(['Comunitaria',
            'Relacionado',
            'En estudio',
            'Importado',],
           ['0',
            '1',
            '2',
            '3',])

df.replace(['casa',
            'Hospital',
            'Hospital UCI',
#   Its important because of the equivalent vallues in other columns,
#   so, in other columns also will be replaced by the same nuemric vallue
            'N/A',
            'Fallecido',
            ],
           ['0',
            '1',
            '2',
            '3',
            '4',])


df.replace(['leve',
            'Moderado',
            'Grave',
#   Its important because of the equivalent vallues in other columns,
#   so, in other columns also will be replaced by the same nuemric vallue
            'N/A',
            'Fallecido',
            ],
           ['0',
            '1',
            '2',
            '3',
            '4',])

'''

In [49]:
df.head(10)

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name
0,2020-03-06,1,2020-03-02,11,BOGOTA,11001,BOGOTA,19,1,F,IMPORTADO,CASA,LEVE,380.0,ITALIA,RECUPERADO,2020-02-27,NaT,2020-03-06,2020-03-13,PCR,6.0,NaN
1,2020-03-09,2,2020-03-06,76,VALLE,76111,BUGA,34,1,M,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-03-04,NaT,2020-03-09,2020-03-19,PCR,5.0,NaN
2,2020-03-09,3,2020-03-07,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-02-29,NaT,2020-03-09,2020-03-15,PCR,6.0,NaN
3,2020-03-11,4,2020-03-09,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,2020-03-06,NaT,2020-03-11,2020-03-26,PCR,6.0,NaN
4,2020-03-11,5,2020-03-09,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,2020-03-08,NaT,2020-03-11,2020-03-23,PCR,6.0,NaN
5,2020-03-11,6,2020-03-10,5,ANTIOQUIA,5360,ITAGUI,27,1,F,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,2020-03-06,NaT,2020-03-11,2020-03-26,PCR,6.0,NaN
6,2020-03-11,7,2020-03-08,13001,CARTAGENA,13001,CARTAGENA,85,1,F,IMPORTADO,CASA,LEVE,840.0,ESTADOS UNIDOS DE AMÉRICA,RECUPERADO,2020-03-02,NaT,2020-03-11,2020-03-17,PCR,6.0,NaN
7,2020-03-11,8,2020-03-09,11,BOGOTA,11001,BOGOTA,22,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-03-06,NaT,2020-03-11,2020-03-21,PCR,6.0,NaN
8,2020-03-11,9,2020-03-08,11,BOGOTA,11001,BOGOTA,28,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-03-07,NaT,2020-03-11,2020-03-23,PCR,6.0,NaN
9,2020-03-12,10,2020-03-12,11,BOGOTA,11001,BOGOTA,36,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-03-06,NaT,2020-03-12,2020-03-21,PCR,6.0,NaN


#Euclidean function



##labeling


*   https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd

*   https://plotly.com/python/knn-classification/




In [28]:
df_to_train = df.iloc[:, lambda df: [3,5,7,9,10,11,12,13,15,16,17,18,20,21]].copy() 
df_to_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4530610 entries, 0 to 4530609
Data columns (total 14 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   davipola_departament_code   int64         
 1   davipola_municipality_code  int64         
 2   age                         int64         
 3   sex                         object        
 4   contagion_type              object        
 5   case_location               object        
 6   state                       object        
 7   iso_country_code            float64       
 8   recuperated                 object        
 9   beginning_symptoms_date     datetime64[ns]
 10  date_of_death               datetime64[ns]
 11  date_of_diagnosis           datetime64[ns]
 12  recovery_type               object        
 13  ethnicity                   float64       
dtypes: datetime64[ns](3), float64(2), int64(3), object(6)
memory usage: 483.9+ MB


In [51]:
for i in df_to_train.select_dtypes(include='object'):
  df_to_train[i] = df_to_train[i].astype('category')
  df_to_train[i+"_cat"] = df_to_train[i].cat.codes

df_to_train

,davipola_departament_code,davipola_municipality_code,age,sex,contagion_type,case_location,state,iso_country_code,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,recovery_type,ethnicity,sex_cat,contagion_type_cat,case_location_cat,state_cat,recuperated_cat,recovery_type_cat
0,11,11001,19,F,IMPORTADO,CASA,LEVE,380.0,RECUPERADO,2020-02-27,NaT,2020-03-06,PCR,6.0,0,2,0,2,2,0
1,76,76111,34,M,IMPORTADO,CASA,LEVE,724.0,RECUPERADO,2020-03-04,NaT,2020-03-09,PCR,5.0,1,2,0,2,2,0
2,5,5001,50,F,IMPORTADO,CASA,LEVE,724.0,RECUPERADO,2020-02-29,NaT,2020-03-09,PCR,6.0,0,2,0,2,2,0
3,5,5001,55,M,RELACIONADO,CASA,LEVE,NaN,RECUPERADO,2020-03-06,NaT,2020-03-11,PCR,6.0,1,3,0,2,2,0
4,5,5001,25,M,RELACIONADO,CASA,LEVE,NaN,RECUPERADO,2020-03-08,NaT,2020-03-11,PCR,6.0,1,3,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4530605,54,54001,46,M,EN ESTUDIO,CASA,LEVE,NaN,ACTIVO,2021-06-25,NaT,2021-07-10,NaN,NaN,1,1,0,2,0,-1
4530606,13,13657,30,M,EN ESTUDIO,CASA,LEVE,NaN,ACTIVO,2021-06-25,NaT,2021-07-10,NaN,NaN,1,1,0,2,0,-1
4530607,54,54874,11,M,EN ESTUDIO,CASA,LEVE,NaN,ACTIVO,2021-06-25,NaT,2021-07-10,NaN,NaN,1,1,0,2,0,-1
4530608,54,54001,29,M,EN ESTUDIO,CASA,LEVE,NaN,ACTIVO,2021-06-25,NaT,2021-07-10,NaN,NaN,1,1,0,2,0,-1


In [ ]:
# df_string = df.loc[:, [
#   "web_date_report",
#   "notification_date",
#   "department_name",
#   "municipality_name",
#   "sex",
#   "contagion_type",
#   "case_location",
#   "state",
#   "country_name",
#   "recuperated",
#   "beginning_symptoms_date",
#   "date_of_diagnosis",
#   "date_of_recovery",
#   "recovery_type",
#   "ethnic_group_name",
#   "date_of_death",
# ]].copy()


In [29]:
#Convert the DataFrame to use best possible dtypes.
dfn = df_to_train.convert_dtypes()
dfn

,davipola_departament_code,davipola_municipality_code,age,sex,contagion_type,case_location,state,iso_country_code,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,recovery_type,ethnicity
0,11,11001,19,F,IMPORTADO,CASA,LEVE,380,RECUPERADO,2020-02-27,NaT,2020-03-06,PCR,6
1,76,76111,34,M,IMPORTADO,CASA,LEVE,724,RECUPERADO,2020-03-04,NaT,2020-03-09,PCR,5
2,5,5001,50,F,IMPORTADO,CASA,LEVE,724,RECUPERADO,2020-02-29,NaT,2020-03-09,PCR,6
3,5,5001,55,M,RELACIONADO,CASA,LEVE,<NA>,RECUPERADO,2020-03-06,NaT,2020-03-11,PCR,6
4,5,5001,25,M,RELACIONADO,CASA,LEVE,<NA>,RECUPERADO,2020-03-08,NaT,2020-03-11,PCR,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4530605,54,54001,46,M,EN ESTUDIO,CASA,LEVE,<NA>,ACTIVO,2021-06-25,NaT,2021-07-10,<NA>,<NA>
4530606,13,13657,30,M,EN ESTUDIO,CASA,LEVE,<NA>,ACTIVO,2021-06-25,NaT,2021-07-10,<NA>,<NA>
4530607,54,54874,11,M,EN ESTUDIO,CASA,LEVE,<NA>,ACTIVO,2021-06-25,NaT,2021-07-10,<NA>,<NA>
4530608,54,54001,29,M,EN ESTUDIO,CASA,LEVE,<NA>,ACTIVO,2021-06-25,NaT,2021-07-10,<NA>,<NA>


In [30]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4530610 entries, 0 to 4530609
Data columns (total 14 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   davipola_departament_code   Int64         
 1   davipola_municipality_code  Int64         
 2   age                         Int64         
 3   sex                         string        
 4   contagion_type              string        
 5   case_location               string        
 6   state                       string        
 7   iso_country_code            Int64         
 8   recuperated                 string        
 9   beginning_symptoms_date     datetime64[ns]
 10  date_of_death               datetime64[ns]
 11  date_of_diagnosis           datetime64[ns]
 12  recovery_type               string        
 13  ethnicity                   Int64         
dtypes: Int64(5), datetime64[ns](3), string(6)
memory usage: 505.5 MB


#Compromiso

1. Servicio que colecta la data por departamento y la guarda en la base de datos
    * Eliminar registros por departamento y volver a consultarlos y almacenarlos
2. Servicio que recibe por parametro el id del departamento 
3. Servicio que recupera la info por municipio.


#Commitment

1. Service which collect the data from Colombia according to each departament (state)
    * Delete records per departament and request it again
2. Service which get by parameter the departament_id and return the information
3. Service get municipality info


#Documentation



request
[location and count](https://datos.gov.co/resource/gt2j-8ykr.json?$select=%20count(id_de_caso),ubicacion&$group=ubicacion)

Json
>```
count_id_de_caso	"11015"
ubicacion	"casa"
count_id_de_caso	"4028443"
ubicacion	"Casa"
count_id_de_caso	"7"
ubicacion	"CASA"
count_id_de_caso	"105326"
ubicacion	"Fallecido"
count_id_de_caso	"22865"
ubicacion	"Hospital"
count_id_de_caso	"4963"
ubicacion	"Hospital UCI"
count_id_de_caso	"14575"
ubicacion	"N/A"
```



Contagion types


**source contagion type**
[request](https://datos.gov.co/resource/gt2j-8ykr.json?$select=%20fuente_tipo_contagio&$group=fuente_tipo_contagio)

Json
>```
[
{"fuente_tipo_contagio":"Comunitaria"}
,{"fuente_tipo_contagio":"En estudio"}
,{"fuente_tipo_contagio":"Importado"}
,{"fuente_tipo_contagio":"Relacionado"}
]
```


Patient State [request](https://datos.gov.co/resource/gt2j-8ykr.json?$select=%20estado&$group=estado)


***Patient state *** 

>```
[{"estado":"Fallecido"}
,{"estado":"Grave"}
,{"estado":"leve"}
,{"estado":"Leve"}
,{"estado":"LEVE"}
,{"estado":"Moderado"}
,{"estado":"N/A"}]
```



#PCA

## Example of PCA

In [34]:
from sklearn.decomposition import PCA

#Example 1
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca = PCA(n_components=2)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

In [ ]:
#Example 2

In [ ]:
pca = PCA(n_components=2, svd_solver='full')
pca.fit(X)

In [ ]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

#Bayesian Algoritm

In [ ]:
#Clustering


Advising by Ian Tomas, after meeting with Michael M. and Sebastian Romero L.

>A good place to start would be a similarity measure in your data, which you can implement with a radial basis function. This function inputs two records from your data set and outputs a single number between 0 and 1 which represents how similar the two records are, where a value closer to 1 means they are more similar, closer to 0 means they are different. This is a good idea to use if you want to represent your data as a graph as typically the similarity measure can work as an edge weight in a graph.

```
 void vectorsubtraction(vector<float> v1, vector<float> v2, vector<float> &v3){

    int i;
    //vectors must be same size
    if(v1.size() != v2.size()) return;
    for(i = 0; i < v1.size(); i++){
        //if symbolic attribute, push 1 to represent not equal, 0 to represent equal
        if(symbolic(i)){
          v3.push_back(v1[i] != v2[i]);
          //if continous, subtract values
        } else {
          v3.push_back((v1[i] - v2[i]));
        }
    }
}

float normalizevector(vector<float> v){

    int i;
    float total = 0.0;
    for(i = 0; i < v.size(); i++){
        total += (v[i] * v[i]);
    }

    return sqrt(total);
}

double edgeweight(vector<float> v1, vector<float> v2){

    vector<float> v3;
    vectorsubtraction(v1, v2, v3);

    float normalized = normalizevector(v3);
    double base = exp(normalized);
    return (1 / base);
}
```



In [ ]:
#Python code of barycentric clustering


In [ ]:
#from sklearn.preprocessing import OneHotEncoder

In [37]:
df_np = df.to_numpy()
print(df_np)
df.info()

KeyboardInterrupt: ignored

In [ ]:
#df_to_use_in_pca = df_to_use_in_pca.apply(pd.to_numeric)

#Clustering

In [42]:
dfn['recuperated']

0          RECUPERADO
1          RECUPERADO
2          RECUPERADO
3          RECUPERADO
4          RECUPERADO
              ...    
4530605        ACTIVO
4530606        ACTIVO
4530607        ACTIVO
4530608        ACTIVO
4530609        ACTIVO
Name: recuperated, Length: 4530610, dtype: string

In [43]:
#for i in df_to_use_in_pca:
#  df_to_use_in_pca[i].astype(int)

enc = OneHotEncoder(handle_unknown='ignore')

enc_df = pd.DataFrame(
    enc.fit_transform(
        dfn['recuperated']).toarray())

# converted = df_to_use_in_pca.to_numpy()
# print(converted)
# enc.fit(converted)

ValueError: ignored

#Label Encoding

In [ ]:
#from sklearn.preprocessing import LabelEncoder

In [ ]:
df.columns

In [33]:
df['case_location_encoded'] = LabelEncoder().fit_transform(df['case_location'])
df[['case_location', 'case_location_encoded']] # special syntax to get just these two columns

TypeError: ignored

In [ ]:
df.head(200)

In [ ]:
print(df.dtypes)

Statistics # Statistics to develop